# L'algoritmo di Dijkstra

L'algoritmo di Dijkstra per trovare l'albero dei cammini minimi è un algoritmo *goloso* molto simile a quello di Prim. Differisce da quest'ultimo rispetto alla funzione di costo, infatti l'obiettivo è quello di **minimizzare il costo del cammino più lungo** che è possibile percorrere a partire da un certo nodo di riferimento.

Quindi, anziché conservare il peso dell'arco di peso minimo per raggiungere un dato nodo, la struttura dati da mantenere è un array che conserva il costo del cammino minimo per raggiungerlo.

<u>Vincolo</u>: i pesi degli archi non possono essere negativi

### Implementazione senza heap

In [7]:
import sys
from grafi.grafinopesati import GrafoNOP, GrafoNOLAP

In [8]:
def dijkstra( g:GrafoNOP ) -> GrafoNOP:
    # Inizializzo le strutture dati di supporto
    presente_sol : list[bool] = [False]*g.n
    pesi_cammini : list[int] = [sys.maxsize]*g.n
    padri : list[int] = [-1]*g.n
    
    ret : GrafoNOP = GrafoNOLAP(g.n) # Albero in uscita
    nodo_rif : int = 0
    presente_sol[nodo_rif] = True
    pesi_cammini[nodo_rif] = 0
    padri[nodo_rif] = 0
    while ( any(not x for x in presente_sol) ): # Finché non ho inserito tutti i nodi nella soluzione
        # Scorro gli adiacenti
        for arco in g.adiacenti(nodo_rif):
            if ( not presente_sol[arco.y] and pesi_cammini[nodo_rif] + arco.peso < pesi_cammini[arco.y] ):
                pesi_cammini[arco.y] = pesi_cammini[nodo_rif] + arco.peso
                padri[arco.y] = nodo_rif
        # Scelgo il prossimo nodo di riferimento (quello il cui costo di raggiungimento è minimo)
        min_peso = min( [pesi_cammini[i] for i in range(g.n) if presente_sol[i] == False] )
        min_nodo = -1
        for i in range(g.n):
            if presente_sol[i] == False and pesi_cammini[i] == min_peso: 
                min_nodo = i 
                break

        x, y = padri[min_nodo], min_nodo
        ret.aggiungiarco(x,y,g.arco(x,y)[2]) # aggiungo l'arco alla soluzione

        nodo_rif = y # aggiorno il nodo di riferimento
        presente_sol[nodo_rif] = True
        
    return ret

In [9]:
# Speriamo funzioni

g : GrafoNOP = GrafoNOLAP(4)
g.aggiungiarco(0,1,1)
g.aggiungiarco(0,3,2)
g.aggiungiarco(1,2,1)
g.aggiungiarco(2,3,1)

alb_cammini_minimi = dijkstra(g)
alb_cammini_minimi.stampa()

(0, 1, 1)
(0, 3, 2)
(1, 0, 1)
(1, 2, 1)
(2, 1, 1)
(3, 0, 2)


Come nel caso di Prim è possibile ridurre la complessità dell'algoritmo da $O(n^2)$ a $O(m \cdot log(n))$ utilizzando un heap modificabile di supporto.

In [11]:
from heap.heap import Heap
from heap.heapmodificabile import HeapModificabile
from grafi.algosugrafipesati import Pair

In [12]:
def prim_migliorato( g:GrafoNOP ) -> GrafoNOP:
    # Inizializzo le strutture dati di supporto
    presente_sol : list[bool] = [False]*g.n
    pesi_cammini : list[int] = [sys.maxsize]*g.n
    padri : list[int] = [-1]*g.n
    heap : HeapModificabile[Pair] = HeapModificabile(g.n)
    
    ret : GrafoNOP = GrafoNOLAP(g.n) # Albero in uscita
    nodo_rif : int = 0
    presente_sol[nodo_rif] = True
    pesi_cammini[nodo_rif] = 0
    padri[nodo_rif] = 0
    
    # Inizializzo l'heap con gli adiacenti di 0
    for arco in g.adiacenti(nodo_rif):
        heap.ins(Pair(arco.y, arco.peso))
        pesi_cammini[arco.y] = arco.peso
        padri[arco.y] = 0 # nodo_rif

    while ( not heap.evuoto() ): # Finché non ho inserito tutti i nodi nella soluzione
        nodo : Pair = heap.out() # Pair ( nodo, arco min per raggiungerlo )
        nodo_rif = nodo.x 

        x, y = padri[nodo_rif], nodo_rif
        ret.aggiungiarco(x,y,g.arco(x,y)[2]) # aggiungo l'arco alla soluzione
        presente_sol[nodo_rif] = True

        # Scorro gli adiacenti
        for arco in g.adiacenti(nodo_rif):
            if ( not presente_sol[arco.y] ):
                a = Pair(arco.y, pesi_cammini[nodo_rif] + arco.peso)
                if ( padri[arco.y] == -1 ):
                    heap.ins(a)
                    pesi_cammini[arco.y] = pesi_cammini[nodo_rif] + arco.peso
                    padri[arco.y] = nodo_rif
                elif ( arco.peso < pesi_cammini[nodo_rif] + arco.peso ):
                    heap.update(a)
                    pesi_cammini[arco.y] = pesi_cammini[nodo_rif] + arco.peso
                    padri[arco.y] = nodo_rif
        
    return ret

In [13]:
# Speriamo funzioni

g : GrafoNOP = GrafoNOLAP(4)
g.aggiungiarco(0,1,1)
g.aggiungiarco(0,3,2)
g.aggiungiarco(1,2,1)
g.aggiungiarco(2,3,1)

alb_cammini_minimi = dijkstra(g)
alb_cammini_minimi.stampa()

(0, 1, 1)
(0, 3, 2)
(1, 0, 1)
(1, 2, 1)
(2, 1, 1)
(3, 0, 2)
